In [19]:
from pydantic import BaseModel
from uuid import UUID
from datetime import datetime
from typing import Protocol, AsyncContextManager, Any, Callable, AsyncIterator, TypeVar, Generic
from redis.asyncio import Redis
from elasticsearch import AsyncElasticsearch
from psycopg  import Cursor
from http import HTTPStatus



### 1. movie service

In [ ]:
# domain
class MovieSchema(BaseModel):
    title        : str
    description  : str | None
    imdb_rating  : float | None
    creation_year: int | None
    
class GenreSchema(BaseModel):
    name       : str
    description: str
    
class PersonSchema(BaseModel):
    full_name: str  
    
class MovieWithRelatedSchema(BaseModel):
    title              : str
    description        : str | None
    imdb_rating        : float | None
    creation_year      : int | None
    genres             : list[GenreSchema]
    genre_names        : list[str]
    actors             : list[PersonSchema]
    actor_full_names   : list[str]
    wtiters            : list[PersonSchema]
    wtiter_full_names  : list[str]
    director           : list[PersonSchema]
    director_full_names: list[str]
    
# infrastructure
class uuidMixin(BaseModel):
    id: UUID
   
class dateMixin(BaseModel):
    created_at: datetime
    updated_at: datetime


class Movie(uuidMixin, dateMixin):
    title: str
    description: str | None
    imdb_rating: float | None
    creation_year: int | None
    
class Genre(uuidMixin, dateMixin):
    name: str
    description: str
    
class Person(uuidMixin, dateMixin):
    full_name: str    
    
class MoviePerson(uuidMixin, dateMixin):
    movie_id: UUID
    person_id: UUID
    role: str  
    
class MovieGenre(uuidMixin, dateMixin):
    movie_id: UUID
    genre_id: UUID  
    

# every storage gvies session object that contains all oeprations
class StorageProtocol(Protocol):
    def session(self) -> Callable[..., AsyncIterator[Cursor | Redis | AsyncElasticsearch]]:
      pass


T = TypeVar("T")

class DatabaseProtocol(Protocol[T]):
    db: Callable[..., AsyncIterator[Cursor | Redis | AsyncElasticsearch]]
    model  : T
    
    def add(self, record: T) -> T:
        pass    
    
    def get_by_id(self, id: UUID) -> T:
        pass
    
    def get_many_with_pagination(self, page:int, per_page:int) -> list[T]:
        pass
    
    def search_with_fulltext_and_pagination(self, key:str, value: str, page:int, per_page:int) -> list[T]:
        pass
    
 
JsonType = None | bool | int | float | str | list["JsonType"] | dict[str, "JsonType"]

class CacheProtocol(Protocol):
    session: Callable[..., AsyncIterator[Cursor | Redis | AsyncElasticsearch]]    
    
    def put(self, key: str, value: JsonType) -> None:
        pass    
    
    def get(self, key: str) -> JsonType:
        pass  

T = TypeVar("T")
  
class BaseRepositoryProtocol(DatabaseProtocol[T], CacheProtocol, Protocol):
    
    async def get_by_id(self, id: UUID) -> T:
        pass
    
    async def get_many_with_sort_and_pagination(self, sort_by:str, sort_order:str, page:str, per_page:str, schema:BaseModel, **kwargs) -> T:
        pass
        
    async def _get_from_db(self, id: UUID) -> T:
        pass

    async def _get_from_cache(self, id: UUID) -> T:
        pass

T = TypeVar("T")

class ServiceProtocol(
    BaseRepositoryProtocol[
        DatabaseProtocol[T], 
        CacheProtocol
    ], 
    Protocol
    ):    
    
    async def get_by_id(self, id: UUID) -> T:
        pass
    
    async def get_many_with_sort_and_pagination(self, sort_by:str, sort_order:str, page:str, per_page:str, schema:BaseModel, **kwargs) -> T:
        pass
    
    async def _make_one_response(self, id: UUID, schema:BaseModel) -> T:
        pass
    
    async def _make_many_response(self, sort_by:str, sort_order:str, page:str, per_page:str, schema:BaseModel, **kwargs) -> T:
        pass
    
class MovieRepository(BaseRepositoryProtocol):
    def __init__(
        self, 
        db: DatabaseProtocol[StorageProtocol, Movie], 
        cache: CacheProtocol
    ) -> None:
        super(DatabaseProtocol[StorageProtocol, Movie]).__init__(db, Movie)
        
class GenreRepository(BaseRepositoryProtocol):
    def __init__(
        self, 
        db: DatabaseProtocol[StorageProtocol, Genre], 
        cache: CacheProtocol
    ) -> None:
        super(DatabaseProtocol[StorageProtocol, Genre]).__init__(db, Genre)
        
class PersonRepository(BaseRepositoryProtocol):
    def __init__(
        self, 
        db: DatabaseProtocol[StorageProtocol, Person], 
        cache: CacheProtocol
    ) -> None:
        super(DatabaseProtocol[StorageProtocol, Person]).__init__(db, Person)


class MovieWithRelatedRespponseSchema(BaseModel):
    data: MovieWithRelatedSchema
    status: HTTPStatus
    
class MovieService:
    def __init__(
        self, 
        movie_repository : MovieRepository,          
    ) -> None:
        self.movie_repository=movie_repository        

    async def get_by_id(self, id: UUID, movie_schema: MovieWithRelatedSchema) -> MovieWithRelatedRespponseSchema:
        movie = await self.movie_repository.get_by_id(id)        
        return  await self._make_one_response(movie_schema, movie)
       
    async def _make_one_response(self, movie_schema, data):        
        if data is not None:
            return MovieWithRelatedRespponseSchema(
                data=movie_schema(**data), 
                status= HTTPStatus.OK
            )
        else:
            return MovieWithRelatedRespponseSchema(
                data=None, 
                status= HTTPStatus.NOT_FOUND
            )
  
# usecases
def get_movie_by_id(id:UUID) -> MovieWithRelatedSchema:
    return ServiceProtocol(
        BaseRepositoryProtocol(
            DatabaseProtocol[StorageProtocol, Any[Movie, Genre, Person],
            CacheProtocol[StorageProtocol]
        )
    ).get_by_id(id, MovieWithRelatedSchema)

    

In [ ]:
1.

### 2. auth service

In [ ]:
class uuidMixin(BaseModel):
    id: UUID
   
class dateMixin(BaseModel):
    created_at: datetime
    updated_at: datetime


class User(uuidMixin, dateMixin):
    first_name: str
    last_name: str | None
    imdb_rating: float | None
    creation_year: int | None
    
class Genre(uuidMixin, dateMixin):
    name: str
    description: str
    
class Person(uuidMixin, dateMixin):
    full_name: str    
    
class MoviePerson(uuidMixin, dateMixin):
    movie_id: UUID
    person_id: UUID
    role: str  
    
class MovieGenre(uuidMixin, dateMixin):
    movie_id: UUID
    genre_id: UUID     
    
    